##### Copyright 2020 TensorFlowAuthors。

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 高度な自動微分

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">GoogleColabで実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

[自動微分ガイドに](autodiff.ipynb)は、勾配の計算に必要なすべてのものが含まれています。このガイドでは、 `tf.GradientTape`より深く一般的ではない機能に焦点を当てています。

## セットアップ

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)

## 勾配記録の制御

[自動微分ガイドで](autodiff.ipynb)は、勾配計算を構築する際に、テープで監視される変数とテンソルを制御する方法を説明しました。

テープには、録音を操作する方法もあります。

If you wish to stop recording gradients, you can use `GradientTape.stop_recording()` to temporarily suspend recording.

これは、モデルの途中で複雑な操作を区別したくない場合に、オーバーヘッドを削減するのに役立つ場合があります。これには、メトリックまたは中間結果の計算が含まれる場合があります。

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  x_sq = x * x
  with t.stop_recording():
    y_sq = y * y
  z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

完全にやり直したい場合は、 `reset()`使用してください。通常、グラデーションテープブロックを終了して再起動するだけの方が読みやすいですが、テープブロックを終了するのが困難または不可能な場合は、 `reset`を使用できます。

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## グラデーションを停止します

上記のグローバルテープコントロールとは対照的に、 `tf.stop_gradient`関数ははるかに正確です。テープ自体にアクセスすることなく、特定のパスに沿ってグラデーションが流れるのを防ぐために使用できます。

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## カスタムグラデーション

場合によっては、デフォルトを使用するのではなく、グラデーションの計算方法を正確に制御したい場合があります。これらの状況は次のとおりです。

- あなたが書いている新しいopのための定義されたグラデーションはありません。
- デフォルトの計算は数値的に不安定です。
- フォワードパスから高価な計算をキャッシュしたいとします。
- ：あなたは（使用例の値を変更したい`tf.clip_by_value` 、 `tf.math.round`勾配を変更せずに）。

For writing a new op, you can use `tf.RegisterGradient` to set up your own. See that page for details. (Note that the gradient registry is global, so change it with caution.)

後者の3つのケースでは、 `tf.custom_gradient`を使用できます。


これは、 `tf.clip_by_norm`を中間グラデーションに適用する例です。

In [ ]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


詳細については、 `tf.custom_gradient`デコレータを参照してください。

## 複数のテープ

複数のテープがシームレスに相互作用します。たとえば、ここでは、各テープが異なるテンソルのセットを監視しています。

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
  tape0.watch(x0)
  tape1.watch(x1)

  y0 = tf.math.sin(x0)
  y1 = tf.nn.sigmoid(x1)

  y = y0 + y1

  ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0).numpy()   # cos(x) => 1.0

In [ ]:
tape1.gradient(ys, x1).numpy()   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25

### 高次勾配

`GradientTape`コンテキストマネージャー内の操作は、自動区別のために記録されます。そのコンテキストで勾配が計算される場合、勾配計算も記録されます。その結果、まったく同じAPIが高次のグラデーションでも機能します。例えば：

In [ ]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    y = x * x * x

  # Compute the gradient inside the outer `t2` context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

これにより*スカラー*関数の2次導関数が得られますが、 `GradientTape.gradient`はスカラーの勾配のみを計算するため、このパターンは一般化されてヘッセ行列を生成しません。ヘッセ行列を作成するには、[ヤコビアンセクションの](#jacobians)[ヘッセ行列の例を](#hessian)参照してください。

"Nested calls to `GradientTape.gradient`" is a good pattern when you are calculating a scalar from a gradient, and then the resulting scalar acts as a source for a second gradient calculation, as in the following example.


#### 例：入力勾配の正則化

Many models are susceptible to "adversarial examples". This collection of techniques modifies the model's input to confuse the model's output. The [simplest implementation](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) takes a single step along the gradient of the output with respect to the input; the "input gradient".

敵対的な例に対するロバスト性を高めるための1つの手法は、[入力勾配の正則化です](https://arxiv.org/abs/1905.11468)。これは、入力勾配の大きさを最小化しようとします。入力勾配が小さい場合、出力の変化も小さいはずです。

以下は、入力勾配正則化の単純な実装です。実装は次のとおりです。

1. 内側のテープを使用して、入力に対する出力の勾配を計算します。
2. その入力勾配の大きさを計算します。
3. モデルに関してその大きさの勾配を計算します。

In [ ]:
x = tf.random.normal([7, 5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
  # The inner tape only takes the gradient with respect to the input,
  # not the variables.
  with tf.GradientTape(watch_accessed_variables=False) as t1:
    t1.watch(x)
    y = layer(x)
    out = tf.reduce_sum(layer(x)**2)
  # 1. Calculate the input gradient.
  g1 = t1.gradient(out, x)
  # 2. Calculate the magnitude of the input gradient.
  g1_mag = tf.norm(g1)

# 3. Calculate the gradient of the magnitude with respect to the model.
dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var.shape for var in dg1_mag]

## ヤコビアン


これまでのすべての例では、いくつかのソーステンソルに関してスカラーターゲットの勾配を取りました。

[ヤコビ行列](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant)は、ベクトル値関数の勾配を表します。各行には、ベクトルの要素の1つの勾配が含まれています。

`GradientTape.jacobian`メソッドを使用すると、ヤコビ行列を効率的に計算できます。

ご了承ください：

- `gradient` ： `sources`引数は、テンソルまたはテンソルのコンテナーにすることができます。
- `gradient`とは異なり： `target`テンソルは単一テンソルでなければなりません。

### スカラーソース

最初の例として、これはスカラーソースに関するベクトルターゲットのヤコビアンです。

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
  y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

スカラーに関してヤコビアンを取ると、結果は**ターゲット**の形状になり、ソースに対する各要素の勾配を示します。

In [ ]:
print(y.shape)
print(dy_dx.shape)

In [ ]:
plt.plot(x.numpy(), y, label='y')
plt.plot(x.numpy(), dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

### テンソルソース

入力がスカラーであるかテンソルであるかにかかわらず、 `GradientTape.jacobian`は、ターゲットの各要素に対するソースの各要素の勾配を効率的に計算します。

たとえば、このレイヤーの出力は`(10, 7)`形状になります。

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
  y = layer(x)

y.shape

And the layer's kernel's shape is `(5, 10)`:

In [ ]:
layer.kernel.shape

カーネルに関する出力のヤコビアンの形状は、連結された2つの形状です。

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

ターゲットの寸法を合計すると、 `GradientTape.gradient`によって計算された合計の勾配が残ります。

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape:', g.shape)

j_sum = tf.reduce_sum(j, axis=[0, 1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()
assert delta < 1e-3
print('delta:', delta)

<a id="hessian"> </a>

#### 例：ヘシアン

`tf.GradientTape`は、ヘッセ行列を構築するための明示的なメソッドを提供していませんが、 `GradientTape.jacobian`メソッドを使用して構築することは可能です。

Note: The Hessian matrix contains `N**2` parameters. For this and other reasons it is not practical for most models. This example is included more as a demonstration of how to use the `GradientTape.jacobian` method, and is not an endorsement of direct Hessian-based optimization. A Hessian-vector product can be [calculated efficiently with nested tapes](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py), and is a much more efficient approach to second-order optimization.


In [ ]:
x = tf.random.normal([7, 5])
layer1 = tf.keras.layers.Dense(8, activation=tf.nn.relu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.relu)

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    x = layer1(x)
    x = layer2(x)
    loss = tf.reduce_mean(x**2)

  g = t1.gradient(loss, layer1.kernel)

h = t2.jacobian(g, layer1.kernel)

In [ ]:
print(f'layer.kernel.shape: {layer1.kernel.shape}')
print(f'h.shape: {h.shape}')

このヘシアンをニュートン法のステップに使用するには、最初にその軸を行列に平坦化し、勾配をベクトルに平坦化します。

In [ ]:
n_params = tf.reduce_prod(layer1.kernel.shape)

g_vec = tf.reshape(g, [n_params, 1])
h_mat = tf.reshape(h, [n_params, n_params])

ヘッセ行列は対称である必要があります。

In [ ]:
def imshow_zero_center(image, **kwargs):
  lim = tf.reduce_max(abs(image))
  plt.imshow(image, vmin=-lim, vmax=lim, cmap='seismic', **kwargs)
  plt.colorbar()

In [ ]:
imshow_zero_center(h_mat)

ニュートン法の更新手順を以下に示します。

In [ ]:
eps = 1e-3
eye_eps = tf.eye(h_mat.shape[0])*eps

注：[実際には行列を反転しないでください](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/)。

In [ ]:
# X(k+1) = X(k) - (∇²f(X(k)))^-1 @ ∇f(X(k))
# h_mat = ∇²f(X(k))
# g_vec = ∇f(X(k))
update = tf.linalg.solve(h_mat + eye_eps, g_vec)

# Reshape the update and apply it to the variable.
_ = layer1.kernel.assign_sub(tf.reshape(update, layer1.kernel.shape))

これは単一の`tf.Variable`場合は比較的簡単ですが、これを自明でないモデルに適用するには、複数の変数にわたって完全なヘッセ行列を生成するために注意深い連結とスライスが必要になります。

### バッチヤコビアン

場合によっては、ソースのスタックに関して、ターゲットのスタックのそれぞれのヤコビアンを取得する必要があります。ここで、各ターゲットとソースのペアのヤコビアンは独立しています。

たとえば、ここでは、入力`x`形状`(batch, ins)`と出力`y`形状`(batch, outs)`です。


In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = layer2(y)

y.shape

The full Jacobian of `y` with respect to `x` has a shape of `(batch, ins, batch, outs)`, even if you only want `(batch, ins, outs)`.

In [ ]:
j = tape.jacobian(y, x)
j.shape

スタック内の各アイテムの勾配が独立している場合、このテンソルのすべての`(batch, batch)`スライスは対角行列です。

In [ ]:
imshow_zero_center(j[:, 0, :, 0])
_ = plt.title('A (batch, batch) slice')

In [ ]:
def plot_as_patches(j):
  # Reorder axes so the diagonals will each form a contiguous patch.
  j = tf.transpose(j, [1, 0, 3, 2])
  # Pad in between each patch.
  lim = tf.reduce_max(abs(j))
  j = tf.pad(j, [[0, 0], [1, 1], [0, 0], [1, 1]],
             constant_values=-lim)
  # Reshape to form a single image.
  s = j.shape
  j = tf.reshape(j, [s[0]*s[1], s[2]*s[3]])
  imshow_zero_center(j, extent=[-0.5, s[2]-0.5, s[0]-0.5, -0.5])

plot_as_patches(j)
_ = plt.title('All (batch, batch) slices are diagonal')

目的の結果を得るには、重複する`batch`ディメンションを合計するか、 `tf.einsum`を使用して対角線を選択します。


In [ ]:
j_sum = tf.reduce_sum(j, axis=2)
print(j_sum.shape)
j_select = tf.einsum('bxby->bxy', j)
print(j_select.shape)

そもそも余分な次元なしで計算を行う方がはるかに効率的です。 `GradientTape.batch_jacobian`メソッドはまさにそれを行います。

In [ ]:
jb = tape.batch_jacobian(y, x)
jb.shape

In [ ]:
error = tf.reduce_max(abs(jb - j_sum))
assert error < 1e-3
print(error.numpy())

Caution: `GradientTape.batch_jacobian` only verifies that the first dimension of the source and target match. It doesn't check that the gradients are actually independent. It's up to the user to ensure they only use `batch_jacobian` where it makes sense. For example adding a `layers.BatchNormalization` destroys the independence, since it normalizes across the `batch` dimension:

In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
bn = tf.keras.layers.BatchNormalization()
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = bn(y, training=True)
  y = layer2(y)

j = tape.jacobian(y, x)
print(f'j.shape: {j.shape}')

In [ ]:
plot_as_patches(j)

_ = plt.title('These slices are not diagonal')
_ = plt.xlabel("Don't use `batch_jacobian`")

In this case `batch_jacobian` still runs and returns _something_ with the expected shape, but it's contents have an unclear meaning.

In [ ]:
jb = tape.batch_jacobian(y, x)
print(f'jb.shape: {jb.shape}')